# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [ ]:
import requests
import time
import re

from random import randint

In [ ]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [ ]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """

    def wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        all_time = end - start
        print(f'Время выполнения функции {func.__name__} в секундах = {all_time}')
        return res

    return wrapper

## Задание 2

In [ ]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """

    def wrapper(*args, **kwargs):
        print(f'Функция вызвана с параметрами:\n {args, kwargs}\n')
        res = func(*args, **kwargs)
        return res

    return wrapper

## Задание 3

In [ ]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """

    def wrapper(*args, **kwargs):
        wrapper.count += 1
        res = func(*args, **kwargs)
        print(f"Функция была вызвана: {wrapper.count} раз\n")
        return res

    wrapper.count = 0
    return wrapper

## Задание 4

In [ ]:
def memo(func):
  """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
  cache = {}

  def fmemo(*args):
      # hash = tuple(args)
      if args not in cache:
          cache[args] = func(*args)
      return cache[args]
  fmemo.cache = cache
  return fmemo

## Тестирование

In [ ]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Функция вызвана с параметрами:
 (('whole',), {})

Время выполнения функции word_count в секундах = 1.003023624420166
Функция была вызвана: 1 раз

Cлово whole встречается 176 раз


In [ ]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [ ]:
%%timeit
fib(3)

724 ns ± 237 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
%%timeit
fib(40)

45.4 s ± 684 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# измеряем время выполнения
@benchmark
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)
fib(3)

Время выполнения функции fib в секундах = 7.152557373046875e-07
Время выполнения функции fib в секундах = 1.1920928955078125e-06
Время выполнения функции fib в секундах = 1.1920928955078125e-06
Время выполнения функции fib в секундах = 0.0007627010345458984
Время выполнения функции fib в секундах = 0.0016026496887207031


2

In [ ]:
@memo
def memo_fib(n):
    if n < 2:
        return n
    return memo_fib(n-2) + memo_fib(n-1)

In [ ]:
%%time
memo_fib(3)

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 11.2 µs


2

In [ ]:
%%time
memo_fib(40)

CPU times: user 49 µs, sys: 1 µs, total: 50 µs
Wall time: 51.7 µs


102334155

In [ ]:
# измеряем время выполнения
@benchmark
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

print(fib(3))

Время выполнения функции fmemo в секундах = 1.1920928955078125e-06
Время выполнения функции fmemo в секундах = 1.430511474609375e-06
Время выполнения функции fmemo в секундах = 9.5367431640625e-07
Время выполнения функции fmemo в секундах = 6.198883056640625e-05
Время выполнения функции fmemo в секундах = 0.0001590251922607422
2


In [ ]:
memo_fib.__closure__[0].cell_contents

{(1,): 1,
 (0,): 0,
 (2,): 1,
 (3,): 2,
 (4,): 3,
 (5,): 5,
 (6,): 8,
 (7,): 13,
 (8,): 21,
 (9,): 34,
 (10,): 55,
 (11,): 89,
 (12,): 144,
 (13,): 233,
 (14,): 377,
 (15,): 610,
 (16,): 987,
 (17,): 1597,
 (18,): 2584,
 (19,): 4181,
 (20,): 6765,
 (21,): 10946,
 (22,): 17711,
 (23,): 28657,
 (24,): 46368,
 (25,): 75025,
 (26,): 121393,
 (27,): 196418,
 (28,): 317811,
 (29,): 514229,
 (30,): 832040,
 (31,): 1346269,
 (32,): 2178309,
 (33,): 3524578,
 (34,): 5702887,
 (35,): 9227465,
 (36,): 14930352,
 (37,): 24157817,
 (38,): 39088169,
 (39,): 63245986,
 (40,): 102334155}

В моих тестах мемоизированная функция при малых числах на входе проигрывала по времени работы немемоизированной. Нано секунды на малых числах простой функции и микросекунды мемоизированной. (Возможно мы ещё сохраняем значения и на расходуем время).

При подаче бОльших чисел в параметры функции получил существенный выигрыш по времени работы. Время вычисления обычной функции уже давало секунды, а мемоизированная функция давала время работы в микросекунды.

(не совсем разобрался можно ли измерять время функций, если применять @benchmark)